In [2]:
# !pip install psycopg2

In [33]:
import pandas as pd
import psycopg2
# Use psycopg2's execute_values for efficient bulk insert
from psycopg2.extras import execute_values
from psycopg2 import sql

In [7]:
# Database connection parameters (replace with actual credentials)
db_params = {
    'dbname': 'smog_db',
    'user': 'smog_dev',
    'password': 'kcSzXTAg1RSXktipRM9gN9cD32pGFN2k',
    'host': 'dpg-csnu6npu0jms73913fh0-a.oregon-postgres.render.com',
    'port': '5432'
}

In [42]:
# Establish the connection
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cursor = conn.cursor()

In [9]:
# Function to list all tables
def list_tables():
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

In [27]:
list_tables()

Tables in the database:
combined_hourly_data


In [12]:
# Read the CSV file
file_path = 'combined_hourly_data.csv'
data = pd.read_csv(file_path)


In [14]:
# Pivot the data to have unique districts as columns
pivot_data = data.pivot_table(index=['date', 'hour'], columns='District', values='AQI', fill_value=None)
pivot_data.head(6)

District          Attock  Bahawalnagar  Bahawalpur  Bhakkar  Chakwal  Chiniot  \
date      hour                                                                  
11/7/2024 0:00   169.505       167.990     180.700  167.380  136.850  277.360   
          10:00  162.065       116.790      93.865  137.380  168.750  178.860   
          11:00  180.930       114.975      96.315  132.650  173.815  182.865   
          12:00  212.610       136.550     119.195  156.255  184.520  227.495   
          13:00  269.325       146.000     141.580  162.205  190.795  242.235   
          14:00  335.830       154.810     169.700  174.735  197.220  276.995   

District         Dera Ghazi Khan  Faisalabad  Gujranwala   Gujrat  ...  \
date      hour                                                     ...   
11/7/2024 0:00           148.595     213.565     201.095  160.745  ...   
          10:00          162.505     167.375     200.315  177.965  ...   
          11:00          174.920     194.325     209.420  178.670  ...   
          12:00          208.065     247.215     229.600  180.725  ...   
          13:00          209.015     263.750     248.915  195.120  ...   
          14:00          235.810     273.910     268.435  210.970  ...   

District         Pakpattan  Rahim Yar Khan  Rajanpur  Rawalpindi  Sahiwal  \
date      hour                                                              
11/7/2024 0:00     159.395         145.730    89.950     151.690  171.440   
          10:00    114.785          79.625   124.615     237.910  118.720   
          11:00    124.290          75.050   124.555     354.910  119.130   
          12:00    150.570          84.030   140.795     440.340  143.260   
          13:00    158.305         103.740   148.160     480.775  151.230   
          14:00    160.780         132.750   157.905     490.430  185.845   

District         Sargodha  Sheikhupura  Sialkot  Toba Tek Singh   Vehari  
date      hour                                                            
11/7/2024 0:00    283.115      534.430  171.055         175.105  191.195  
          10:00   189.580      227.235  185.625         124.455   92.865  
          11:00   198.745      287.775  199.255         132.065  100.970  
          12:00   220.160      359.035  215.845         160.360  125.380  
          13:00   232.700      394.835  230.255         164.455  147.885  
          14:00   252.445      418.280  266.585         163.085  174.055  

[6 rows x 36 columns]

In [15]:
# Reset the index to flatten the DataFrame
pivot_data.reset_index(inplace=True)

In [16]:
pivot_data.head(6)

District,date,hour,Attock,Bahawalnagar,Bahawalpur,Bhakkar,Chakwal,Chiniot,Dera Ghazi Khan,Faisalabad,...,Pakpattan,Rahim Yar Khan,Rajanpur,Rawalpindi,Sahiwal,Sargodha,Sheikhupura,Sialkot,Toba Tek Singh,Vehari
0,11/7/2024,0:00,169.505,167.990,180.700,167.380,136.850,277.360,148.595,213.565,...,159.395,145.730,89.950,151.690,171.440,283.115,534.430,171.055,175.105,191.195
1,11/7/2024,10:00,162.065,116.790,93.865,137.380,168.750,178.860,162.505,167.375,...,114.785,79.625,124.615,237.910,118.720,189.580,227.235,185.625,124.455,92.865
2,11/7/2024,11:00,180.930,114.975,96.315,132.650,173.815,182.865,174.920,194.325,...,124.290,75.050,124.555,354.910,119.130,198.745,287.775,199.255,132.065,100.970
3,11/7/2024,12:00,212.610,136.550,119.195,156.255,184.520,227.495,208.065,247.215,...,150.570,84.030,140.795,440.340,143.260,220.160,359.035,215.845,160.360,125.380
4,11/7/2024,13:00,269.325,146.000,141.580,162.205,190.795,242.235,209.015,263.750,...,158.305,103.740,148.160,480.775,151.230,232.700,394.835,230.255,164.455,147.885
5,11/7/2024,14:00,335.830,154.810,169.700,174.735,197.220,276.995,235.810,273.910,...,160.780,132.750,157.905,490.430,185.845,252.445,418.280,266.585,163.085,174.055


In [19]:
# Get the list of columns (date, hour, and unique district names)
columns = pivot_data.columns.tolist()
columns

['date',
 'hour',
 'Attock',
 'Bahawalnagar',
 'Bahawalpur',
 'Bhakkar',
 'Chakwal',
 'Chiniot',
 'Dera Ghazi Khan',
 'Faisalabad',
 'Gujranwala',
 'Gujrat',
 'Hafizabad',
 'Jhang',
 'Jhelum',
 'Kasur',
 'Khanewal',
 'Khushab',
 'Lahore',
 'Layyah',
 'Lodhran',
 'Mandi Bahuddin',
 'Mianwali',
 'Multan',
 'Muzaffargarh',
 'Nankana Sahib',
 'Narowal',
 'Okara',
 'Pakpattan',
 'Rahim Yar Khan',
 'Rajanpur',
 'Rawalpindi',
 'Sahiwal',
 'Sargodha',
 'Sheikhupura',
 'Sialkot',
 'Toba Tek Singh',
 'Vehari']

In [26]:
# Create table with date, hour, and unique district columns
create_table_query = sql.SQL(
    "CREATE TABLE IF NOT EXISTS combined_hourly_data ("
    "date DATE, "
    "hour INTEGER, "
    + ", ".join([f'"{col}" FLOAT' for col in columns[2:]])
    + ");"
)

# Execute the query to create the table
cursor.execute(create_table_query)


In [32]:
# Alter the table to change the hour column to TIME with explicit casting
alter_table_query = """
    ALTER TABLE combined_hourly_data
    ALTER COLUMN hour TYPE TIME USING (LPAD(hour::text, 2, '0') || ':00')::time;
"""

cursor.execute(alter_table_query)


In [40]:
# Create the INSERT query with quoted column names
insert_query = sql.SQL(
    "INSERT INTO combined_hourly_data ({}) VALUES %s"
).format(
    sql.SQL(", ").join([sql.Identifier(col) for col in columns])
)


In [35]:
# Prepare data for insertion
values = [tuple(row) for row in pivot_data.to_numpy()]

In [36]:
values

[('11/7/2024',
  '0:00',
  169.505,
  167.99,
  180.7,
  167.38,
  136.85,
  277.36,
  148.595,
  213.565,
  201.095,
  160.745,
  275.16,
  223.765,
  156.205,
  232.96,
  193.295,
  239.42,
  247.08,
  188.81,
  191.32,
  237.505,
  161.57,
  201.285,
  200.26,
  216.76,
  147.595,
  194.27,
  159.395,
  145.73,
  89.95,
  151.69,
  171.44,
  283.115,
  534.43,
  171.055,
  175.105,
  191.195),
 ('11/7/2024',
  '10:00',
  162.065,
  116.79,
  93.865,
  137.38,
  168.75,
  178.86,
  162.505,
  167.375,
  200.315,
  177.965,
  203.36,
  132.775,
  175.985,
  157.445,
  102.015,
  168.935,
  236.435,
  134.33,
  85.065,
  198.84,
  145.34,
  113.59,
  133.645,
  164.345,
  165.075,
  139.925,
  114.785,
  79.625,
  124.615,
  237.91,
  118.72,
  189.58,
  227.235,
  185.625,
  124.455,
  92.865),
 ('11/7/2024',
  '11:00',
  180.93,
  114.975,
  96.315,
  132.65,
  173.815,
  182.865,
  174.92,
  194.325,
  209.42,
  178.67,
  206.005,
  138.875,
  173.295,
  161.9,
  102.72,
  166.21,
 

In [41]:
execute_values(cursor, insert_query, values)

In [52]:
# Establish the connection
conn = psycopg2.connect(**db_params)
conn.autocommit = True
cursor = conn.cursor()

In [50]:
# Query to show the top 2 rows in the table
def show_top_rows():
    
    query = "SELECT * FROM combined_hourly_data LIMIT 25;"
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

# Call the function to display the top 2 rows
show_top_rows()


(datetime.date(2024, 11, 7), datetime.time(0, 0), 169.505, 167.99, 180.7, 167.38, 136.85, 277.36, 148.595, 213.565, 201.095, 160.745, 275.16, 223.765, 156.205, 232.96, 193.295, 239.42, 247.08, 188.81, 191.32, 237.505, 161.57, 201.285, 200.26, 216.76, 147.595, 194.27, 159.395, 145.73, 89.95, 151.69, 171.44, 283.115, 534.43, 171.055, 175.105, 191.195)
(datetime.date(2024, 11, 7), datetime.time(10, 0), 162.065, 116.79, 93.865, 137.38, 168.75, 178.86, 162.505, 167.375, 200.315, 177.965, 203.36, 132.775, 175.985, 157.445, 102.015, 168.935, 236.435, 134.33, 85.065, 198.84, 145.34, 113.59, 133.645, 164.345, 165.075, 139.925, 114.785, 79.625, 124.615, 237.91, 118.72, 189.58, 227.235, 185.625, 124.455, 92.865)
(datetime.date(2024, 11, 7), datetime.time(11, 0), 180.93, 114.975, 96.315, 132.65, 173.815, 182.865, 174.92, 194.325, 209.42, 178.67, 206.005, 138.875, 173.295, 161.9, 102.72, 166.21, 315.475, 139.855, 86.67, 203.645, 142.47, 124.81, 139.3, 169.335, 170.72, 140.04, 124.29, 75.05, 124.555

In [53]:
# Execute the ALTER TABLE command to add a unique constraint on date and hour
alter_table_query = """
    ALTER TABLE combined_hourly_data
    ADD CONSTRAINT unique_date_hour UNIQUE (date, hour);
"""

try:
    cursor.execute(alter_table_query)
    conn.commit()
    print("Unique constraint on (date, hour) added successfully.")
except psycopg2.Error as e:
    print("An error occurred:", e)
finally:
    cursor.close()
    conn.close()


Unique constraint on (date, hour) added successfully.


In [51]:
def get_last_24_hours(district):
    
    # Establish the connection
    conn = psycopg2.connect(**db_params)
    conn.autocommit = True
    cursor = conn.cursor()
    
    query = sql.SQL("""
        SELECT date, hour, {}
        FROM combined_hourly_data
        WHERE {} IS NOT NULL
        ORDER BY date DESC, hour DESC
        LIMIT 24;
    """).format(sql.Identifier(district), sql.Identifier(district))

    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

# Example usage
get_last_24_hours('Attock')


(datetime.date(2024, 11, 9), datetime.time(23, 0), 229.545)
(datetime.date(2024, 11, 9), datetime.time(22, 0), 238.355)
(datetime.date(2024, 11, 9), datetime.time(21, 0), 246.835)
(datetime.date(2024, 11, 9), datetime.time(20, 0), 256.665)
(datetime.date(2024, 11, 9), datetime.time(19, 0), 264.6)
(datetime.date(2024, 11, 9), datetime.time(18, 0), 277.78)
(datetime.date(2024, 11, 9), datetime.time(17, 0), 289.79)
(datetime.date(2024, 11, 9), datetime.time(16, 0), 301.815)
(datetime.date(2024, 11, 9), datetime.time(15, 0), 309.32)
(datetime.date(2024, 11, 9), datetime.time(14, 0), 293.135)
(datetime.date(2024, 11, 9), datetime.time(13, 0), 179.29)
(datetime.date(2024, 11, 9), datetime.time(12, 0), 227.38)
(datetime.date(2024, 11, 9), datetime.time(11, 0), 198.79)
(datetime.date(2024, 11, 9), datetime.time(10, 0), 171.33)
(datetime.date(2024, 11, 9), datetime.time(9, 0), 160.99)
(datetime.date(2024, 11, 9), datetime.time(8, 0), 178.42)
(datetime.date(2024, 11, 9), datetime.time(7, 0), 203

In [54]:
def nnn(num):
    reverse = num[::-1]
    return reverse


nnn(1223)

TypeError: 'int' object is not subscriptable